## Step 1: Mount Google Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

# Create directories
!mkdir -p /content/drive/MyDrive/quantum_trader/models
!mkdir -p /content/drive/MyDrive/quantum_trader/data
!mkdir -p /content/drive/MyDrive/quantum_trader/logs

print("✅ Drive mounted successfully")

ModuleNotFoundError: No module named 'google.colab'

## Step 2: Install Dependencies

In [ ]:
%%capture
!pip install TA-Lib yfinance pandas numpy scikit-learn joblib matplotlib seaborn plotly

print("✅ Dependencies installed")

## Step 3: Configuration

In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
import talib
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import joblib
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Configuration
TICKERS = ['MU', 'IONQ', 'APLD', 'ANNX', 'AAPL', 'MSFT', 'NVDA', 'TSLA', 'GOOGL', 'AMZN', 
           'META', 'AMD', 'AVGO', 'MRVL', 'CRM', 'DDOG', 'PLTR', 'COIN', 'JPM', 'XOM']
TRAINING_PERIOD = '3y'  # 3 years for robust models
FORWARD_DAYS = 7  # Predict 7-day returns
DRIVE_PATH = '/content/drive/MyDrive/quantum_trader'

print(f"Training {len(TICKERS)} tickers with {TRAINING_PERIOD} lookback")
print(f"Forward prediction window: {FORWARD_DAYS} days")

## Step 4: Feature Engineering Class

In [ ]:
class FeatureEngineer:
    """Create 50+ ML features from OHLCV data."""
    
    @staticmethod
    def engineer(df: pd.DataFrame) -> pd.DataFrame:
        feat = pd.DataFrame(index=df.index)
        
        # Price arrays
        close = df['Close'].values
        high = df['High'].values
        low = df['Low'].values
        volume = df['Volume'].values
        open_price = df['Open'].values
        
        # Momentum
        feat['rsi_9'] = talib.RSI(close, timeperiod=9)
        feat['rsi_14'] = talib.RSI(close, timeperiod=14)
        feat['rsi_21'] = talib.RSI(close, timeperiod=21)
        
        # MACD
        macd, macd_signal, macd_hist = talib.MACD(close, fastperiod=5, slowperiod=13, signalperiod=1)
        feat['macd'] = macd
        feat['macd_signal'] = macd_signal
        feat['macd_hist'] = macd_hist
        
        # Stochastics
        slowk, slowd = talib.STOCH(high, low, close, fastk_period=14, slowk_period=3, slowd_period=3)
        feat['stoch_k'] = slowk
        feat['stoch_d'] = slowd
        
        # Trend
        feat['adx'] = talib.ADX(high, low, close, timeperiod=14)
        feat['cci'] = talib.CCI(high, low, close, timeperiod=14)
        feat['aroon_up'], feat['aroon_down'] = talib.AROON(high, low, timeperiod=14)
        
        # Volatility
        feat['atr'] = talib.ATR(high, low, close, timeperiod=14)
        feat['natr'] = talib.NATR(high, low, close, timeperiod=14)
        feat['bb_upper'], feat['bb_middle'], feat['bb_lower'] = talib.BBANDS(close, timeperiod=20)
        feat['bb_pct'] = (close - feat['bb_lower']) / (feat['bb_upper'] - feat['bb_lower'])
        
        # Moving averages
        for period in [5, 8, 13, 21, 34, 55, 89]:
            feat[f'ema_{period}'] = talib.EMA(close, timeperiod=period)
            feat[f'sma_{period}'] = talib.SMA(close, timeperiod=period)
        
        # Price vs EMAs
        feat['price_vs_ema5'] = (close - feat['ema_5']) / close
        feat['price_vs_ema13'] = (close - feat['ema_13']) / close
        feat['price_vs_ema21'] = (close - feat['ema_21']) / close
        
        # Volume
        feat['obv'] = talib.OBV(close, volume)
        feat['ad'] = talib.AD(high, low, close, volume)
        feat['adosc'] = talib.ADOSC(high, low, close, volume, fastperiod=3, slowperiod=10)
        feat['vol_sma_20'] = talib.SMA(volume, timeperiod=20)
        feat['vol_ratio'] = volume / feat['vol_sma_20']
        
        # Patterns
        feat['doji'] = talib.CDLDOJI(open_price, high, low, close)
        feat['engulfing'] = talib.CDLENGULFING(open_price, high, low, close)
        feat['hammer'] = talib.CDLHAMMER(open_price, high, low, close)
        feat['shooting_star'] = talib.CDLSHOOTINGSTAR(open_price, high, low, close)
        
        # Returns
        feat['returns_1d'] = close / np.roll(close, 1) - 1
        feat['returns_3d'] = close / np.roll(close, 3) - 1
        feat['returns_7d'] = close / np.roll(close, 7) - 1
        
        return feat.dropna()

print("✅ FeatureEngineer class defined")

## Step 5: Label Creation

In [ ]:
def create_labels(df: pd.DataFrame, forward_days: int = 7) -> pd.Series:
    """
    Labels: 0=BEARISH (< -2%), 1=NEUTRAL (-2% to +2%), 2=BULLISH (> +2%)
    """
    close = df['Close'].values
    future_returns = (np.roll(close, -forward_days) / close - 1) * 100
    
    labels = np.where(future_returns < -2, 0, np.where(future_returns > 2, 2, 1))
    return pd.Series(labels, index=df.index)

print("✅ Label function defined")

## Step 6: Training Pipeline

In [ ]:
def train_ticker_model(ticker: str, period: str = '3y'):
    """
    Full training pipeline:
    1. Fetch data from yfinance
    2. Engineer features
    3. Walk-forward cross-validation
    4. Train final calibrated model
    5. Save to Google Drive
    """
    print(f"\n{'='*60}")
    print(f"Training: {ticker}")
    print(f"{'='*60}")
    
    # Fetch data
    df = yf.download(ticker, period=period, interval='1d', progress=False, auto_adjust=True)
    
    if len(df) < 300:
        print(f"⚠️  Insufficient data for {ticker} ({len(df)} rows)")
        return None
    
    print(f"Data: {len(df)} rows from {df.index[0].date()} to {df.index[-1].date()}")
    
    # Feature engineering
    fe = FeatureEngineer()
    X = fe.engineer(df)
    y = create_labels(df, forward_days=FORWARD_DAYS)
    
    # Align
    common_idx = X.index.intersection(y.index)
    X = X.loc[common_idx]
    y = y.loc[common_idx]
    
    print(f"Features: {X.shape[1]}, Samples: {len(X)}")
    print(f"Label distribution: {y.value_counts().sort_index().to_dict()}")
    
    # Walk-forward validation (5 folds)
    tscv = TimeSeriesSplit(n_splits=5)
    fold_scores = []
    
    for fold, (train_idx, test_idx) in enumerate(tscv.split(X), 1):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
        
        # Scale
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)
        
        # Train base model
        base_model = HistGradientBoostingClassifier(
            max_iter=200,
            learning_rate=0.05,
            max_depth=6,
            min_samples_leaf=20,
            l2_regularization=0.1,
            random_state=42
        )
        
        # Calibrate probabilities
        calibrated = CalibratedClassifierCV(base_model, method='sigmoid', cv=3)
        calibrated.fit(X_train_scaled, y_train)
        
        # Evaluate
        y_pred = calibrated.predict(X_test_scaled)
        acc = accuracy_score(y_test, y_pred)
        fold_scores.append(acc)
        
        print(f"  Fold {fold}: Accuracy = {acc:.3f} (Train: {len(X_train)}, Test: {len(X_test)})")
    
    avg_acc = np.mean(fold_scores)
    std_acc = np.std(fold_scores)
    print(f"\n✅ Walk-forward validation: {avg_acc:.3f} ± {std_acc:.3f}")
    
    # Train final model on ALL data
    scaler_final = StandardScaler()
    X_scaled = scaler_final.fit_transform(X)
    
    final_base = HistGradientBoostingClassifier(
        max_iter=200,
        learning_rate=0.05,
        max_depth=6,
        min_samples_leaf=20,
        l2_regularization=0.1,
        random_state=42
    )
    final_model = CalibratedClassifierCV(final_base, method='sigmoid', cv=3)
    final_model.fit(X_scaled, y)
    
    # Feature importance
    if hasattr(final_model.estimators_[0], 'feature_importances_'):
        importances = final_model.estimators_[0].feature_importances_
        top_features = sorted(zip(X.columns, importances), key=lambda x: x[1], reverse=True)[:10]
        print("\nTop 10 features:")
        for feat, imp in top_features:
            print(f"  {feat:<20}: {imp:.4f}")
    
    # Save model
    model_data = {
        'model': final_model,
        'scaler': scaler_final,
        'feature_columns': X.columns.tolist(),
        'accuracy_mean': avg_acc,
        'accuracy_std': std_acc,
        'ticker': ticker,
        'training_samples': len(X),
        'training_period': period,
        'forward_days': FORWARD_DAYS,
        'trained_datetime': datetime.now().isoformat(),
        'label_distribution': y.value_counts().to_dict()
    }
    
    model_path = f"{DRIVE_PATH}/models/{ticker}_tuned.pkl"
    joblib.dump(model_data, model_path)
    print(f"💾 Model saved: {model_path}")
    
    return {
        'ticker': ticker,
        'accuracy': avg_acc,
        'std': std_acc,
        'samples': len(X),
        'features': X.shape[1]
    }

print("✅ Training pipeline defined")

## Step 7: Train All Models (RUN THIS CELL)

In [ ]:
# Train all tickers
results = []

for ticker in TICKERS:
    try:
        result = train_ticker_model(ticker, period=TRAINING_PERIOD)
        if result:
            results.append(result)
    except Exception as e:
        print(f"❌ Error training {ticker}: {e}")

# Summary
print("\n" + "="*60)
print("TRAINING COMPLETE")
print("="*60)

results_df = pd.DataFrame(results)
results_df = results_df.sort_values('accuracy', ascending=False)
print(results_df.to_string(index=False))

print(f"\nAverage accuracy: {results_df['accuracy'].mean():.3f}")
print(f"Best model: {results_df.iloc[0]['ticker']} ({results_df.iloc[0]['accuracy']:.3f})")
print(f"\n💾 All models saved to: {DRIVE_PATH}/models/")

## Step 8: Download Models to Local System

**Instructions:**

1. After training completes, navigate to Google Drive:
   - Go to `My Drive → quantum_trader → models/`
   - You'll see all `.pkl` files (e.g., `MU_tuned.pkl`, `IONQ_tuned.pkl`, etc.)

2. Download all `.pkl` files to your local system:
   - Select all `.pkl` files
   - Right-click → Download
   - Save to: `E:\quantum-ai-trader-v1.1\models\`

3. Verify local models:
   ```powershell
   ls E:\quantum-ai-trader-v1.1\models\*.pkl
   ```

4. Start local orchestrator:
   ```powershell
   cd E:\quantum-ai-trader-v1.1
   python trading_orchestrator.py
   ```

---

## Step 9: Backtest Performance (Optional)

Test models on recent unseen data:

In [ ]:
def backtest_model(ticker: str):
    """Test model on last 60 days (unseen data)."""
    model_path = f"{DRIVE_PATH}/models/{ticker}_tuned.pkl"
    model_data = joblib.load(model_path)
    
    model = model_data['model']
    scaler = model_data['scaler']
    
    # Fetch recent data
    df = yf.download(ticker, period='90d', interval='1d', progress=False, auto_adjust=True)
    
    fe = FeatureEngineer()
    X = fe.engineer(df)
    y = create_labels(df, forward_days=FORWARD_DAYS)
    
    common_idx = X.index.intersection(y.index)
    X = X.loc[common_idx]
    y = y.loc[common_idx]
    
    # Use last 30 days only
    X_test = X.tail(30)
    y_test = y.tail(30)
    
    X_test_scaled = scaler.transform(X_test)
    y_pred = model.predict(X_test_scaled)
    
    acc = accuracy_score(y_test, y_pred)
    print(f"{ticker}: {acc:.3f} accuracy on last 30 days")
    
    return acc

# Backtest all
print("Backtesting on recent unseen data...\n")
backtest_accs = []
for ticker in TICKERS[:5]:  # Test first 5
    try:
        acc = backtest_model(ticker)
        backtest_accs.append(acc)
    except Exception as e:
        print(f"Error: {e}")

print(f"\nAverage backtest accuracy: {np.mean(backtest_accs):.3f}")

## 🎯 DEPLOYMENT CHECKLIST

After training completes:

- [ ] Download all `.pkl` files from Google Drive
- [ ] Copy to `E:\quantum-ai-trader-v1.1\models\`
- [ ] Verify file count matches number of tickers
- [ ] Run `python trading_orchestrator.py` locally
- [ ] Check logs for model loading confirmation
- [ ] Monitor first 5-minute cycle for signal generation
- [ ] Review production logs in `data/production.db`

**Weekly Retraining:**
- Re-run this notebook every Sunday
- Download updated models
- Replace old `.pkl` files locally
- Restart orchestrator

---

✅ **Training pipeline ready for 24/7 operation!**